In [ ]:
from libs.env import *

from datetime import datetime as dt

from configs.apis import APIS

from libs.agents import Agent, AgentConfig, create_agent, invoke_agent
from libs.base import Directories,Timestamp
from libs.graphs import run_team_workflow
from libs.io import read_text, write_text
# from rich import print as richprint
from IPython.display import Markdown as md
# from flask.views import F

dirs = Directories()
log_prefix = "log_"
filename = log_prefix + Timestamp().date + ".joblib"
log_file_path = os.path.join(dirs.logs, filename)

# print_heading("Available APIs and Models",'green')
# eprint(MODELS)
# print("\n")

print_heading("Available Agent Personas", "green")
eprint([x for x in list(AGENTS.keys())])
print("\n")

# print_heading("Project Directories", "green")
# print_dict(dirs.__dict__, "green")

In [ ]:
### Load daily response log and display last response

try:
    log = read_file(log_file_path)
    try:
        display(md(log[-1]['response']))
    except:
        print(log)
except:
    print("DAILY LOG NOT FOUND")



### Testing local LLMs

In [ ]:
import os
os.environ['HF_HOME'] = 'code/hub'

# Use this for databricks?
# os.environ['TRANSFORMERS_CACHE'] = '/dbfs/hugging_face_transformers_cache/'

import transformers
import torch
hf_token = APIS["huggingface"]["key"].get_secret_value()
# model_name = "meta-llama/Meta-Llama-3-8B"

# pipeline = transformers.pipeline(
#     "text-generation", 
#     model=model_name, 
#     model_kwargs={"torch_dtype": torch.bfloat16}, 
#     device_map="auto",
#     token=hf_token,
#     )
# pipeline("Hey how are you doing today?")


# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Qwen/Qwen2-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name,token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_name,token=hf_token)


In [ ]:
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
device = "cuda" # the device to load the model onto

os.environ['HF_HOME'] = 'M:/Code/resources/hub'


model_name = "google/gemma-2-9b
tokenizer = AutoTokenizer.from_pretrained(model_name,token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_name,token=hf_token)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10,token=hf_token)
hf = HuggingFacePipeline(pipeline=pipe)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    # "Qwen/Qwen2-72B-Instruct",
    "google/gemma-2-9b",
    torch_dtype="auto",
    device_map="auto",
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-72B-Instruct")

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
import os
os.chdir('code/llama3')
import fire
from llama import Dialog, Llama
from langchain_nomic.embeddings import NomicEmbeddings
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate


ckpt_dir = 'M:/Code/llama3/Meta-Llama-3-8B-Instruct/'
tokenizer_path = 'M:/Code/llama3/Meta-Llama-3-8B-Instruct/tokenizer.model'
max_seq_len = 1024
max_batch_size = 64

# generator = Llama.build(
#         ckpt_dir=ckpt_dir,
#         tokenizer_path=tokenizer_path,
#         max_seq_len=max_seq_len,
#         max_batch_size=max_batch_size,
#     )

local_llm = llama3

# LLM
llm = ChatOllama(model=local_llm, format="json", temperature=0)

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance 
    of a retrieved document to a user question. If the document contains keywords related to the user question, 
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()
